# 👥 Group Information

1. **Mahmudur Rahman Mehraj** — ID: 2022-3-60-182  
2. **MD. Robiul Islam** — ID: 2023-1-60-093  
3. **Sunzid Ashraf Mahi** — ID: 2023-1-60-148


# Clear Current Output Directory

In [ ]:
!rm -rf /kaggle/working/*

# Copy Dataset to working Directory

In [ ]:
input_data_path = '/kaggle/input/object-detection-mendeley-yolo-v11/Object Detection Dataset  Navigation Assistance for the Visually Impaired People using YOLOv11/'
input_data_path2 = '/kaggle/input/object-detection-ewu/'
!cp -r "{input_data_path2}"* /kaggle/working/

data_path = '/kaggle/working'


In [ ]:
# Installations
!pip install ttach
!pip install ultralytics --no-deps
!git clone https://github.com/rigvedrs/YOLO-V12-CAM.git
%cd /kaggle/working/YOLO-V12-CAM

In [ ]:
# Imports
import os
import cv2
import yaml
import random
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
import matplotlib.pyplot as plt
from yolo_cam.eigen_cam import EigenCAM
from yolo_cam.utils.image import show_cam_on_image, scale_cam_image


# YAML data loading

In [ ]:
yaml_path = data_path + '/data.yaml'
output_yaml_path = '/kaggle/working/data.yaml'

with open(yaml_path, 'r') as file:
    data = yaml.safe_load(file)

data['train'] = data_path + '/train/images'
data['val'] = data_path + '/valid/images'
data['test'] = data_path + '/test/images'

with open(output_yaml_path, 'w') as file:
    yaml.dump(data, file, default_flow_style=False)

print("Updated data.yaml:")
print(yaml.dump(data, default_flow_style=False))

In [ ]:
def inspect_label_format(split_name="train", num_files=3):
    """Inspect the format of label files"""
    base_path = Path(data_path)
    labels_path = base_path / split_name / "labels"

    print(f"🔍 Inspecting label format in '{split_name}' split:\n")

    label_files = list(labels_path.glob("*.txt"))[:num_files]

    for label_file in label_files:
        print(f"📄 File: {label_file.name}")
        with open(label_file, 'r') as f:
            lines = f.readlines()[:3]
            for i, line in enumerate(lines, 1):
                parts = line.strip().split()
                print(f"   Line {i}: {len(parts)} values -> {line.strip()}")
        print()

inspect_label_format()

# Visualization of YOLO dataset with polygon segmentation masks

In [ ]:
def visualize_yolo_polygon_split(split_name, num_images=3):

    base_path = Path(data_path)
    images_path = base_path / split_name / "images"
    labels_path = base_path / split_name / "labels"

    # Class names
    class_names = ['Bin', 'Building Pillar', 'Bus', 'CNG', 'Car', 'Cycle',
                   'Electric Pole', 'Food Van', 'Food cart', 'Footpath', 'Leguna',
                   'Motorcycle', 'Obstacle', 'Parking Cone', 'Person', 'Pickup',
                   'Rickshaw', 'Stairs', 'Tree', 'Truck', 'Van', 'Van gari']

    print(f"\n📸 Showing {num_images} sample images from '{split_name}' split\n")

    # Get image files
    image_files = list(images_path.glob("*.jpg")) + list(images_path.glob("*.png"))
    image_files = image_files[:num_images]

    # Create subplots
    fig, axes = plt.subplots(1, len(image_files), figsize=(6*len(image_files), 6))
    if len(image_files) == 1:
        axes = [axes]

    for idx, img_file in enumerate(image_files):
        # Read image
        img = cv2.imread(str(img_file))
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img.shape[:2]

        # Read corresponding label
        label_file = labels_path / f"{img_file.stem}.txt"

        if label_file.exists():
            with open(label_file, 'r') as f:
                lines = f.readlines()

            for line in lines:
                parts = list(map(float, line.strip().split()))

                if len(parts) >= 3:  # At least class + 1 point (x,y)
                    cls = int(parts[0])

                    # Extract polygon coordinates (all values after class ID)
                    coords = parts[1:]

                    # Reshape into (N, 2) array of (x, y) points
                    num_points = len(coords) // 2
                    polygon_points = np.array(coords[:num_points * 2]).reshape(-1, 2)

                    # Convert normalized coordinates to pixel coordinates
                    polygon_points[:, 0] *= w
                    polygon_points[:, 1] *= h
                    polygon_points = polygon_points.astype(np.int32)

                    # Generate random color for this object
                    color = tuple(np.random.randint(100, 255, 3).tolist())

                    # Draw filled polygon (semi-transparent)
                    overlay = img.copy()
                    cv2.fillPoly(overlay, [polygon_points], color)
                    img = cv2.addWeighted(img, 0.7, overlay, 0.3, 0)

                    # Draw polygon outline
                    cv2.polylines(img, [polygon_points], True, color, 2)

                    # Add label at the top-left of the polygon
                    label = class_names[cls] if cls < len(class_names) else f"Class {cls}"
                    min_point = polygon_points.min(axis=0)
                    cv2.putText(img, label, tuple(min_point),
                               cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Display
        axes[idx].imshow(img)
        axes[idx].axis('off')
        axes[idx].set_title(f"{img_file.name}\n({w}x{h})")

    plt.tight_layout()
    plt.show()

# Visualize each split
print("=" * 60)
print("TRAIN SPLIT")
print("=" * 60)
visualize_yolo_polygon_split("train", num_images=3)

print("\n" + "=" * 60)
print("VALIDATION SPLIT")
print("=" * 60)
visualize_yolo_polygon_split("valid", num_images=3)

print("\n" + "=" * 60)
print("TEST SPLIT")
print("=" * 60)
visualize_yolo_polygon_split("test", num_images=3)

# Load Dataset YAML

In [ ]:
# ------------------------------------------------------------
# Load Dataset YAML
# ------------------------------------------------------------
yaml_path = Path(data_path + '/data.yaml')

with open(yaml_path, "r") as f:
    data_cfg = yaml.safe_load(f)

print("✅ Dataset loaded successfully:")
print(yaml.dump(data_cfg, sort_keys=False))


In [ ]:
# ------------------------------------------------------------
# Hyper Parameters
# ------------------------------------------------------------
epoch = 50
batch_size = 64
image_size = 640
learning_rate = 0.01
learning_rate_fine = 0.001
patience = 15
optimizer = 'AdamW'
IOU = 0.7

# YOLO v11
## Model Training

In [ ]:
# ------------------------------------------------------------
# Training
# ------------------------------------------------------------
model = YOLO("yolo11n.pt")

results = model.train(
    data=str(yaml_path),
    epochs=epoch,              # ← More epochs for 22 classes
    patience=patience,             # ← Longer patience for complex task
    imgsz=image_size,               # Or 800 if GPU memory allows
    batch=batch_size,                # ← Reduced batch for better gradient updates
    project="/kaggle/working",
    name="yolov11",
    device=0,
    workers=2,
    exist_ok=True,
    
    # Learning rate - important for multi-class
    lr0=learning_rate,
    lrf=learning_rate_fine,               # ← Lower final LR for fine-tuning

    
    # Stronger augmentation for 22 classes
    hsv_h=0.015,             # HSV-Hue augmentation
    hsv_s=0.7,               # HSV-Saturation
    hsv_v=0.4,               # HSV-Value
    degrees=20.0,            # Rotation
    translate=0.2,           # Translation
    scale=0.7,               # Scale
    shear=2.0,               # Shear
    perspective=0.0001,      # Perspective
    flipud=0.0,              # No vertical flip
    fliplr=0.5,              # Horizontal flip
    mosaic=1.0,              # Mosaic (great for multi-class)
    mixup=0.2,               # Mixup
    copy_paste=0.5,          # ← Higher for rare classes
    
    # Multi-class optimization
    optimizer=optimizer,
    weight_decay=0.0005,
    momentum=0.937,
    close_mosaic=10,         # Disable mosaic in last 10 epochs
    
    # Class-specific settings
    cls=0.5,                 # ← Class loss weight (important!)
    box=7.5,                 # Box loss weight
    dfl=1.5,                 # DFL loss weight
    
    # NMS settings for multiple classes
    iou=IOU,                 # IoU threshold
    conf=0.001,              # Low conf during training
    
    # Validation
    val=True,
    save=True,
    save_period=25,
    plots=True,
    
    # Performance
    amp=True,
    rect=False,              # Rectangular training (can help)
    
    # Verbose
    verbose=True,
)

print("\n✅ Training complete. Best model saved in:")
print(model.ckpt_path)

## Evaluation on Validation and Test Sets

In [ ]:
# ------------------------------------------------------------
# Evaluation on Validation and Test Sets
# ------------------------------------------------------------
print("\n📊 Evaluating on validation set...")
val_metrics = model.val(data=str(yaml_path), split="val")
print("\nValidation Results:")
# print(val_metrics)

print("\n📊 Evaluating on test set...")
test_metrics = model.val(data=str(yaml_path), split="test")
print("\nTest Results:")
# print(test_metrics)

## Training Predictions Visualization

In [ ]:
# ------------------------------------------------------------
# Training Predictions Visualization
# ------------------------------------------------------------
print("\n🧠 Running inference for visualization...")

val_path = Path((data_cfg["val"])[0:])
print(f"Looking for images in: {val_path}")

sample_imgs = list(val_path.glob("*.jpg")) + list(val_path.glob("*.png"))

if not sample_imgs:
    print(f"❌ No images found in {val_path}")
    print("Available files:")
    print(list(val_path.glob("*"))[:10])
else:
    print(f"✅ Found {len(sample_imgs)} images")

    num_samples = min(10, len(sample_imgs))
    sample_imgs = random.sample(sample_imgs, num_samples)

    for img_path in sample_imgs:
        print(f"\n📸 Processing: {img_path.name}")

        results = model(str(img_path))
        annotated = results[0].plot() 

        plt.figure(figsize=(10, 10))
        plt.imshow(cv2.cvtColor(annotated, cv2.COLOR_BGR2RGB))
        plt.axis("off")
        plt.title(f"Predicted: {img_path.name}", fontsize=14, fontweight='bold')
        plt.tight_layout()
        plt.show()

        boxes = results[0].boxes
        if len(boxes) > 0:
            print(f"   Detected {len(boxes)} objects")
            for box in boxes:
                cls_id = int(box.cls[0])
                conf = float(box.conf[0])
                cls_name = model.names[cls_id]
                print(f"   - {cls_name}: {conf:.2%}")
        else:
            print("   No objects detected")

## Loss and mAP Curves 

In [ ]:
# ------------------------------------------------------------
# Loss and mAP Curves 
# ------------------------------------------------------------
results_csv = '/kaggle/working/yolov11/results.csv'

if not os.path.exists(results_csv):
    print(f"❌ results.csv not found at {results_csv}")
else:
    # Load metrics
    df = pd.read_csv(results_csv)
    print(f"\n✅ Loaded training metrics from: {results_csv}")
    print(f"Available columns: {df.columns.tolist()}")
    
    # Clean column names (remove leading/trailing spaces)
    df.columns = df.columns.str.strip()
    
    # Create output directory
    output_dir = '/kaggle/working/yolov11/training_plots'
    os.makedirs(output_dir, exist_ok=True)
    
    # ------------------------------------------------------------
    # Plot 1: Loss curves
    # ------------------------------------------------------------
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df["train/box_loss"], label="Box Loss", color="red", linewidth=2)
    plt.plot(df.index, df["train/cls_loss"], label="Class Loss", color="blue", linewidth=2)
    plt.plot(df.index, df["train/dfl_loss"], label="DFL Loss", color="green", linewidth=2)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Loss", fontsize=12)
    plt.title("YOLOv11 Training Loss Curves", fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    
    # Save to working directory (shows up in Kaggle output)
    loss_plot_path = os.path.join(output_dir, 'training_loss_curves.png')
    plt.savefig(loss_plot_path, dpi=150, bbox_inches='tight')
    print(f"✅ Saved loss plot to: {loss_plot_path}")
    plt.show()
    plt.close()
    
    # ------------------------------------------------------------
    # Plot 2: mAP curves
    # ------------------------------------------------------------
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5", 
             color="orange", linewidth=2, marker='o', markersize=3)
    plt.plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95", 
             color="purple", linewidth=2, marker='s', markersize=3)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("mAP", fontsize=12)
    plt.title("YOLOv11 Validation mAP Curves", fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.tight_layout()
    
    # Save to working directory
    map_plot_path = os.path.join(output_dir, 'validation_mAP_curves.png')
    plt.savefig(map_plot_path, dpi=150, bbox_inches='tight')
    print(f"✅ Saved mAP plot to: {map_plot_path}")
    plt.show()
    plt.close()
    
    # ------------------------------------------------------------
    # Plot 3: Combined metrics (Precision, Recall, mAP)
    # ------------------------------------------------------------
    plt.figure(figsize=(12, 6))
    plt.plot(df.index, df["metrics/precision(B)"], label="Precision", 
             color="green", linewidth=2, marker='^', markersize=3)
    plt.plot(df.index, df["metrics/recall(B)"], label="Recall", 
             color="red", linewidth=2, marker='v', markersize=3)
    plt.plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5", 
             color="blue", linewidth=2, marker='o', markersize=3)
    plt.xlabel("Epoch", fontsize=12)
    plt.ylabel("Score", fontsize=12)
    plt.title("YOLOv11 Validation Metrics", fontsize=14, fontweight='bold')
    plt.legend(fontsize=11)
    plt.grid(True, linestyle="--", alpha=0.5)
    plt.ylim([0, 1])  # Set y-axis from 0 to 1
    plt.tight_layout()
    
    # Save to working directory
    metrics_plot_path = os.path.join(output_dir, 'validation_metrics.png')
    plt.savefig(metrics_plot_path, dpi=150, bbox_inches='tight')
    print(f"✅ Saved metrics plot to: {metrics_plot_path}")
    plt.show()
    plt.close()
    
    # ------------------------------------------------------------
    # Plot 4: All losses in one subplot
    # ------------------------------------------------------------
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    
    # Training losses
    axes[0, 0].plot(df.index, df["train/box_loss"], color="red", linewidth=2)
    axes[0, 0].set_title("Box Loss", fontweight='bold')
    axes[0, 0].set_xlabel("Epoch")
    axes[0, 0].grid(True, alpha=0.3)
    
    axes[0, 1].plot(df.index, df["train/cls_loss"], color="blue", linewidth=2)
    axes[0, 1].set_title("Class Loss", fontweight='bold')
    axes[0, 1].set_xlabel("Epoch")
    axes[0, 1].grid(True, alpha=0.3)
    
    axes[1, 0].plot(df.index, df["train/dfl_loss"], color="green", linewidth=2)
    axes[1, 0].set_title("DFL Loss", fontweight='bold')
    axes[1, 0].set_xlabel("Epoch")
    axes[1, 0].grid(True, alpha=0.3)
    
    # mAP comparison
    axes[1, 1].plot(df.index, df["metrics/mAP50(B)"], label="mAP@0.5", 
                    color="orange", linewidth=2)
    axes[1, 1].plot(df.index, df["metrics/mAP50-95(B)"], label="mAP@0.5:0.95", 
                    color="purple", linewidth=2)
    axes[1, 1].set_title("mAP Scores", fontweight='bold')
    axes[1, 1].set_xlabel("Epoch")
    axes[1, 1].legend()
    axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    
    # Save combined plot
    combined_plot_path = os.path.join(output_dir, 'training_summary.png')
    plt.savefig(combined_plot_path, dpi=150, bbox_inches='tight')
    print(f"✅ Saved combined plot to: {combined_plot_path}")
    plt.show()
    plt.close()
    
    # ------------------------------------------------------------
    # Save summary statistics
    # ------------------------------------------------------------
    summary_path = os.path.join(output_dir, 'training_summary.txt')
    with open(summary_path, 'w') as f:
        f.write("YOLOv11 Training Summary\n")
        f.write("=" * 60 + "\n\n")
        f.write(f"Total Epochs: {len(df)}\n\n")
        
        f.write("Final Metrics:\n")
        f.write("-" * 60 + "\n")
        f.write(f"Train Box Loss: {df['train/box_loss'].iloc[-1]:.4f}\n")
        f.write(f"Train Class Loss: {df['train/cls_loss'].iloc[-1]:.4f}\n")
        f.write(f"Train DFL Loss: {df['train/dfl_loss'].iloc[-1]:.4f}\n\n")
        
        f.write(f"Validation Precision: {df['metrics/precision(B)'].iloc[-1]:.4f}\n")
        f.write(f"Validation Recall: {df['metrics/recall(B)'].iloc[-1]:.4f}\n")
        f.write(f"Validation mAP@0.5: {df['metrics/mAP50(B)'].iloc[-1]:.4f}\n")
        f.write(f"Validation mAP@0.5:0.95: {df['metrics/mAP50-95(B)'].iloc[-1]:.4f}\n\n")
        
        f.write("Best Metrics:\n")
        f.write("-" * 60 + "\n")
        f.write(f"Best mAP@0.5: {df['metrics/mAP50(B)'].max():.4f} at epoch {df['metrics/mAP50(B)'].idxmax()}\n")
        f.write(f"Best mAP@0.5:0.95: {df['metrics/mAP50-95(B)'].max():.4f} at epoch {df['metrics/mAP50-95(B)'].idxmax()}\n")
        f.write(f"Best Precision: {df['metrics/precision(B)'].max():.4f} at epoch {df['metrics/precision(B)'].idxmax()}\n")
        f.write(f"Best Recall: {df['metrics/recall(B)'].max():.4f} at epoch {df['metrics/recall(B)'].idxmax()}\n")
    
    print(f"✅ Saved training summary to: {summary_path}")
    
    print(f"\n{'='*60}")
    print(f"📁 All plots and summary saved to: {output_dir}")
    print(f"{'='*60}")
    print(f"\nFiles created:")
    print(f"  1. training_loss_curves.png")
    print(f"  2. validation_mAP_curves.png")
    print(f"  3. validation_metrics.png")
    print(f"  4. training_summary.png")
    print(f"  5. training_summary.txt")


## XAI


In [ ]:
%cd /kaggle/working/YOLO-V12-CAM

In [ ]:
model = YOLO(data_path + '/yolov11/weights/best.pt')
model = model.cpu()

In [ ]:
img = cv2.imread('/kaggle/input/object-detection-ewu/train/images/frame_0s_jpg.rf.05d81773433c10ba035e8c1c7a6e8407.jpg')
img = cv2.resize(img, (640, 640))
rgb_img = img.copy()
img = np.float32(img) / 255

In [ ]:
target_layers =[model.model.model[-2]]

In [ ]:
cam = EigenCAM(model, target_layers,task='od')
grayscale_cam = cam(rgb_img)[0, :, :]
cam_image = show_cam_on_image(img, grayscale_cam, use_rgb=True)
plt.imshow(cam_image)
plt.show()

In [ ]:
g_scale = np.stack([grayscale_cam] * 3, axis=2)
plt.imshow(g_scale)